In [ ]:
# ===============================
# AI Fashion Design Generator (Improved DCGAN)
# ===============================
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output

# ------------------------------
# 1. Load & Prepare Data
# ------------------------------
(X_train, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
X_train = (X_train - 127.5) / 127.5  # Normalize to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1).astype('float32')

BUFFER_SIZE = 60000
BATCH_SIZE = 128
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# ------------------------------
# 2. Build Generator
# ------------------------------
def make_generator():
    model = tf.keras.Sequential([
        layers.Input(shape=(100,)),
        layers.Dense(7*7*256, use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# ------------------------------
# 3. Build Discriminator
# ------------------------------
def make_discriminator():
    model = tf.keras.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (5,5), strides=(2,2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5,5), strides=(2,2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# ------------------------------
# 4. Define Loss and Optimizers
# ------------------------------
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator = make_generator()
discriminator = make_discriminator()

generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

# ------------------------------
# 5. Training Step
# ------------------------------
@tf.function
def train_step(images):
    current_batch = tf.shape(images)[0]  # dynamically detect batch size
    noise = tf.random.normal([current_batch, 100])
    real_labels = tf.ones((current_batch, 1)) * 0.9  # label smoothing
    fake_labels = tf.zeros((current_batch, 1))

    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        d_loss_real = cross_entropy(real_labels, real_output)
        d_loss_fake = cross_entropy(fake_labels, fake_output)
        d_loss = d_loss_real + d_loss_fake

        g_loss = cross_entropy(real_labels, fake_output)

    gradients_of_generator = gen_tape.gradient(g_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(d_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return g_loss, d_loss
# ------------------------------
# 6. Training Loop
# ------------------------------
EPOCHS = 15
noise_seed = tf.random.normal([16, 100])
os.makedirs("outputs", exist_ok=True)

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :, 0] + 1) / 2.0, cmap='gray')
        plt.axis('off')
    plt.suptitle(f'Epoch {epoch}', fontsize=14)
    plt.savefig(f'outputs/fashion_epoch_{epoch:03d}.png')
    plt.close(fig)

def train(dataset, epochs):
    for epoch in range(1, epochs + 1):
        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
        clear_output(wait=True)
        print(f'Epoch {epoch}/{epochs} — g_loss: {g_loss:.4f}, d_loss: {d_loss:.4f}')
        generate_and_save_images(generator, epoch, noise_seed)

train(dataset, EPOCHS)
print("\n✅ Training complete! Check 'outputs' folder for generated designs.")

Epoch 15/15 — g_loss: 0.9211, d_loss: 1.2859

✅ Training complete! Check 'outputs' folder for generated designs.


In [ ]:
!ls outputs

fashion_epoch_001.png  fashion_epoch_006.png  fashion_epoch_011.png
fashion_epoch_002.png  fashion_epoch_007.png  fashion_epoch_012.png
fashion_epoch_003.png  fashion_epoch_008.png  fashion_epoch_013.png
fashion_epoch_004.png  fashion_epoch_009.png  fashion_epoch_014.png
fashion_epoch_005.png  fashion_epoch_010.png  fashion_epoch_015.png
